# Requirements

In [29]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import joblib 
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime 
import pickle

# 회귀분석
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from math import sqrt

import warnings
warnings.filterwarnings("ignore")

# Data Load

In [48]:
data_dir = './datasets/raw_data'

In [49]:
data_list = os.listdir(data_dir)
print(len(data_list))
print(data_list)

40
['2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2016.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2017.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2018.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2019.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인타자_2020.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2016.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2017.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2018.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2019.csv', '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2020.csv', '2020빅콘테스트_스포츠투아이_제공데이터_경기_2016.csv', '2020빅콘테스트_스포츠투아이_제공데이터_경기_2017.csv', '2020빅콘테스트_스포츠투아이_제공데이터_경기_2018.csv', '2020빅콘테스트_스포츠투아이_제공데이터_경기_2019.csv', '2020빅콘테스트_스포츠투아이_제공데이터_경기_2020.csv', '2020빅콘테스트_스포츠투아이_제공데이터_등록선수_2016.csv', '2020빅콘테스트_스포츠투아이_제공데이터_등록선수_2017.csv', '2020빅콘테스트_스포츠투아이_제공데이터_등록선수_2018.csv', '2020빅콘테스트_스포츠투아이_제공데이터_등록선수_2019.csv', '2020빅콘테스트_스포츠투아이_제공데이터_등록선수_2020.csv', '2020빅콘테스트_스포츠투아이_제공데이터_선수_2016.csv', '2020빅콘테스트_스포츠투아이_제공데이터_선수_2017.csv', '2020빅콘테스트_스포츠투아이_제공데이터_선수_2018.csv', '2020빅콘테스트_스포츠투아이_제공데이터_선수_2019.csv', '2020빅콘테스트_스포츠투아이_제공데이터_선수_2020.csv', '2020빅콘테스트_스포츠투아

In [50]:
#년도별 저장
data_list_2016 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2016') ]
data_2016 = [pd.read_csv(os.path.join(data_dir,data_list_2016[x]),encoding='cp949') for x in range(len(data_list_2016))]
data_list_2017 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2017') ]
data_2017 = [pd.read_csv(os.path.join(data_dir,data_list_2017[x]),encoding='cp949') for x in range(len(data_list_2017))]
data_list_2018 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2018') ]
data_2018 = [pd.read_csv(os.path.join(data_dir,data_list_2018[x]),encoding='cp949') for x in range(len(data_list_2018))]
data_list_2019 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2019') ]
data_2019 = [pd.read_csv(os.path.join(data_dir,data_list_2019[x]),encoding='cp949') for x in range(len(data_list_2019))]
data_list_2020 = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-1].startswith('2020') ]
data_2020 = [pd.read_csv(os.path.join(data_dir,data_list_2020[x]),encoding='cp949') for x in range(len(data_list_2020))]
print('2020년 데이터: \n', data_2020)

#항목별 저장
data_list_single_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인타자')]
data_single_hitter = [pd.read_csv(os.path.join(data_dir, data_list_single_hitter[x]), encoding='cp949') for x in range(len(data_list_single_hitter))]
data_list_single_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('개인투수')]
data_single_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_single_pitcher[x]), encoding='cp949') for x in range(len(data_list_single_pitcher))]
data_list_games = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('경기')]
data_games = [pd.read_csv(os.path.join(data_dir, data_list_games[x]), encoding='cp949') for x in range(len(data_list_games))]
data_list_player_enroll = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('등록선수')]
data_player_enroll = [pd.read_csv(os.path.join(data_dir, data_list_player_enroll[x]), encoding='cp949') for x in range(len(data_list_player_enroll))]
data_list_players = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('선수')]
data_players = [pd.read_csv(os.path.join(data_dir, data_list_players[x]), encoding='cp949') for x in range(len(data_list_players))]
data_list_teams = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀')]
data_teams = [pd.read_csv(os.path.join(data_dir, data_list_teams[x]), encoding='cp949') for x in range(len(data_list_teams))]
data_list_team_hitter = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀타자')]
data_team_hitter = [pd.read_csv(os.path.join(data_dir, data_list_team_hitter[x]), encoding='cp949') for x in range(len(data_list_team_hitter))]
data_list_team_pitcher = [data_list[x] for x in range(len(data_list)) if data_list[x].split('_')[-2].startswith('팀투수')]
data_team_pitcher = [pd.read_csv(os.path.join(data_dir, data_list_team_pitcher[x]), encoding='cp949') for x in range(len(data_list_team_pitcher))]
print('팀투수 데이터:\n', data_team_pitcher)

2020년 데이터: 
 [               G_ID   GDAY_DS T_ID VS_T_ID  HEADER_NO TB_SC   P_ID  START_CK  \
0     20200505HHSK0  20200505   HH      SK          0     T  61208         1   
1     20200505HHSK0  20200505   SK      HH          0     B  61353         1   
2     20200505HHSK0  20200505   HH      SK          0     T  62700         1   
3     20200505HHSK0  20200505   SK      HH          0     B  62895         1   
4     20200505HHSK0  20200505   SK      HH          0     B  63450         1   
...             ...       ...  ...     ...        ...   ...    ...       ...   
8055  20200719WOSK0  20200719   SK      WO          0     B  75847         1   
8056  20200719WOSK0  20200719   SK      WO          0     B  76802         1   
8057  20200719WOSK0  20200719   SK      WO          0     B  77463         0   
8058  20200719WOSK0  20200719   WO      SK          0     T  78168         1   
8059  20200719WOSK0  20200719   WO      SK          0     T  79456         1   

      BAT_ORDER_NO  PA  .

# Add_data

In [51]:
add_pitcher = pd.read_csv('./datasets/pitcher_final.csv')
add_hitter = pd.read_csv('./datasets/hitter_final.csv')

#2020년 데이터에 추가
data_team_pitcher[-1] = pd.concat([data_team_pitcher[-1],add_pitcher])

data_team_hitter[-1] = pd.concat([data_team_hitter[-1],add_hitter])


# Util

In [6]:
def make_y_next(df, rm_range, target):
    df['y_next'] = 0
    
    t_dict = {}
    t_list = []
    count = 0
    f_count = 0
    for i, v in enumerate(df[target]):

        if df[target].index[i] == df[target].index[-1]:
            t_dict[str(f_count)] = round(np.mean(np.array(t_list)),5)

        t_list.append(v)
        count += 1

        if count == rm_range:
            t_dict[str(f_count)] = round(np.mean(np.array(t_list)),5)
            count = 0
            t_list = []
            f_count += 1

    rm_dict = {}
    rm_list = []
    rm_count = 0
    rm_fcount = 0

    for i, v in enumerate(df['y_next']):
        if df['y_next'].index[i] == df['y_next'].index[-1]:
            rm_dict[str(rm_fcount)] = rm_list

        rm_list.append(i)
        rm_count += 1

        if rm_count == rm_range:
            rm_dict[str(rm_fcount)] = rm_list
            rm_count = 0
            rm_list = []
            rm_fcount +=1

    for k, v in rm_dict.items():
        try:
            df['y_next'].iloc[v] = t_dict[str(int(k)+1)]
        except: 
            df['y_next'].iloc[v] = np.nan
            
    return df

# Outlier

In [7]:
hitter_outlier = ['20190825WOSS0','20170702WOKT0','20170930WONC0','20190831WONC0','20180811HTSK0','20170708HTKT0','20170521WOKT0',
                  '20190630HTKT0','20170719NCHH0','20180904LGKT0','20180613SKHT0','20170702HTLG0','20170617SKSS0','20170702SSSK0',
                  '20160831NCKT0','20170917OBSS0','20190823NCLG0','20200625NCKT2','20190515KTHT0','20180915NCOB0','20190820HTLG0',
                  '20170919KTLG0','20170818HHNC0','20200610HTKT0','20190928NCKT0','20190407HHLT0']

pitcher_outlier = ['20170702WOKT0','20200705LGSS0','20190608LTKT0','20180426OBSK0','20180422WOHH0','20160408HTKT0','20190521OBKT0',
                   '20170502OBSS0','20170702HTLG0','20170702SSSK0','20200512HTHH0','20170616HHKT0','20180610WOKT0','20160619NCKT0',
                   '20180529KTSS0','20200610HTKT0','20200602WOHH0','20180523OBHH0','20190407HHLT0','20160505OBLG0']
hitter_outlier = [line.strip() for line in hitter_outlier]
pithcer_outlier = [line.strip() for line in pitcher_outlier]

# LGBM

## EDA

### ERA

In [8]:
from sklearn.preprocessing import LabelEncoder

def lgbm_pitcher_eda(year, team_name):
    year_index = year - 2016 
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가

    data_team_pitcher_year = data_team_pitcher[year_index]

    data_team_pitcher_year_team = data_team_pitcher_year[data_team_pitcher_year.T_ID == team_name]
    df = data_team_pitcher_year_team

    df['TB_SC'] = df['TB_SC'].map(lambda x: 1 if x == 'B' else 0 )

    df = df.drop(columns = ['GDAY_DS'])
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    df['WLS'] = encoder.fit_transform(df['WLS']) #승패무 여부
    
    
    ### 이부분은 모든 EDA에서 동일하게 작성
    
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']

    #df['G_ID'] = pd.Series(dt)
    df['year']=df['G_ID'].str.slice(0,4).astype(int)
    df['month']=df['G_ID'].str.slice(4,6).astype(int)
    
    df["FIP"] = ((13*df.HR+3*(df.BB+df.HP-df.IB)-2*df.KK)/(df.INN2/3)) +3.2
    df["BABIP"] = (df.HIT-df.HR)/(df.AB-df.KK-df.HR+df.SF)
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']

    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('G_ID')

    # 경기당 방어율 column 생성
    df['ERA'] = df['ER'] * 27 / df['INN2']
    
    df['avg_ERA'] = df['ERA'].mean()
    df['ERA_trend'] = (df['ERA'] - df['avg_ERA']).astype(np.float16)
    df.drop(['avg_ERA'],axis=1,inplace=True)

    df = df.drop(columns=['T_ID','P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT'])
    df_year_team_name = df
    
    return df_year_team_name

### AVG

In [9]:
from sklearn.preprocessing import LabelEncoder

def lgbm_hitter_eda(year, team_name):
    year_index = year - 2016 
    
    # team_hitter data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_team_hitter_year = data_team_hitter[year_index]

    data_team_hitter_year_team = data_team_hitter_year[data_team_hitter_year.T_ID == team_name]
    df = data_team_hitter_year_team

    df['TB_SC'] = df['TB_SC'].map(lambda x: 1 if x == 'B' else 0 )

    df = df.drop(columns = ['GDAY_DS'])
    
    df.reset_index(inplace=True, drop=True) #index 재정렬
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    

    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']
    #장타율
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    #출루율
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']
    
    df = df.drop(columns=['T_ID'])
    
    # 경기당 타율 column 생성
    df['AVG'] = df['HIT'] / df['AB']
    
    df['avg_AVG'] = df['AVG'].mean()
    df['AVG_trend'] = (df['AVG'] - df['avg_AVG']).astype(np.float16)
    df.drop(['avg_AVG'],axis=1,inplace=True)
    
    df_year_team_name = df
    
    return df_year_team_name

## Modeling

In [10]:
# 파라미터 지정

lgb_params_grid = {
    'num_leaves': [5,10,20,30],
    'min_data_in_leaf': [10,30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}

svr_params_grid = { 
    'kernel':['rbf','poly'],
    'degree':[2,3,4,5,6,7],
    'epsilon':[0.1,0.2,1,10,20,30]
}

rf_params_grid = {
    'n_estimators': [10, 100],
    'max_depth': [6,8,10,12],
    'min_samples_leaf': [8,12,18],
    'min_samples_split': [8,16,20]
}

### ERA

In [11]:
class LGBM_pitcher:
    def __init__(self, eda,  team, yn_range, model_type):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
        self.model_type = model_type
    
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        
        df_all = df_all[df_all.index.isin(pithcer_outlier)==False]
        df_all =df_all.reset_index()
        df_all = make_y_next(df_all, self.yn_range, 'ERA')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]

        if(self.model_type=="lgbm"):
            m = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
            m_grid = GridSearchCV(estimator=m,param_grid=lgb_params_grid,n_jobs=10,verbose=3)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            
        elif(self.model_type=="rf"):
            m = RandomForestRegressor(random_state=0, n_jobs=1)
            m_grid = GridSearchCV(m, param_grid = rf_params_grid, cv = 3, n_jobs = -1)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            

        elif(self.model_type=="SVR"):
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            m = SVR(kernel='rbf')
            m_grid = GridSearchCV(estimator=m, param_grid=svr_params_grid) 
            m_grid.fit(X_train, y_train)
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_test = scaler.fit_transform(final_test)
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()

        return final_predict_mean

### AVG

In [12]:
class LGBM_hitter:
    def __init__(self, eda,  team, yn_range, model_type):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
        self.model_type = model_type
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)

        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all = df_all[df_all.index.isin(hitter_outlier)==False]
        df_all =df_all.reset_index()
        
        df_all = make_y_next(df_all, self.yn_range, 'AVG')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 
        df_all.dropna(inplace=True)
        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train test val split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]
        

        if(self.model_type=="lgbm"):
            m = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
            m_grid = GridSearchCV(estimator=m,param_grid=lgb_params_grid,n_jobs=10,verbose=3)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            
        elif(self.model_type=="rf"):
            m = RandomForestRegressor(random_state=0, n_jobs=1)
            m_grid = GridSearchCV(m, param_grid = rf_params_grid, cv = 3, n_jobs = -1)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            

        elif(self.model_type=="SVR"):
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            m = SVR(kernel='rbf')
            m_grid = GridSearchCV(estimator=m, param_grid=svr_params_grid) 
            m_grid.fit(X_train, y_train)
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_test = scaler.fit_transform(final_test)
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()

        return final_predict_mean

### 승률

In [13]:
class LGBM_lose:
    def __init__(self, eda,  team, yn_range, model_type):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
        self.model_type = model_type
        
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)
        
        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all = df_all[df_all.index.isin(pithcer_outlier)==False]
        df_all =df_all.reset_index()

        df_all = make_y_next(df_all, self.yn_range, 'R')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]

        if(self.model_type=="lgbm"):
            m = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
            m_grid = GridSearchCV(estimator=m,param_grid=lgb_params_grid,n_jobs=10,verbose=3)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            
        elif(self.model_type=="rf"):
            m = RandomForestRegressor(random_state=0, n_jobs=1)
            m_grid = GridSearchCV(m, param_grid = rf_params_grid, cv = 3, n_jobs = -1)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            

        elif(self.model_type=="SVR"):
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            m = SVR(kernel='rbf')
            m_grid = GridSearchCV(estimator=m, param_grid=svr_params_grid) 
            m_grid.fit(X_train, y_train)
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_test = scaler.fit_transform(final_test)
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()

        return final_predict

In [14]:
class LGBM_score:
    def __init__(self, eda,  team, yn_range,model_type):
        self.eda = eda
        self.team = team
        self.yn_range = yn_range
        self.model_type = model_type
        
    def run(self):
        df_2016 = self.eda(2016, self.team)
        df_2017 = self.eda(2017, self.team)
        df_2018 = self.eda(2018, self.team)
        df_2019 = self.eda(2019, self.team)
        df_2020 = self.eda(2020, self.team)

        df_all = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020])
        df_all = df_all[df_all.index.isin(hitter_outlier)==False]
        df_all =df_all.reset_index()
        
        df_all = make_y_next(df_all, self.yn_range, 'RUN')
        
        # 테스트용 데이터 뽑기
        final_test = df_all[df_all['y_next'].isnull()]; 

        df_all.dropna(inplace=True)

        df_all=df_all.drop(['G_ID'],axis=1)

        # 범주형 변수 지정
        cat_features = ['VS_T_ID' ,'HEADER_NO', 'TB_SC' ]
        df_all[cat_features] = df_all[cat_features].astype('category')

        # X,y Train test val split
        X = df_all.drop(columns = ['y_next'])
        y = df_all['y_next']

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, shuffle = True, random_state = 2020)

        X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
        X_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_val.columns]


        if(self.model_type=="lgbm"):
            m = LGBMRegressor(boosting_type='gbdt', num_boost_round=2000, learning_rate=0.01)
            m_grid = GridSearchCV(estimator=m,param_grid=lgb_params_grid,n_jobs=10,verbose=3)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            
        elif(self.model_type=="rf"):
            m = RandomForestRegressor(random_state=0, n_jobs=1)
            m_grid = GridSearchCV(m, param_grid = rf_params_grid, cv = 3, n_jobs = -1)
            m_grid.fit(X_train,y_train) 
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()
            

        elif(self.model_type=="SVR"):
            scaler = MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            m = SVR(kernel='rbf')
            m_grid = GridSearchCV(estimator=m, param_grid=svr_params_grid) 
            m_grid.fit(X_train, y_train)
            m = m_grid
            
            final_test = final_test.drop(['G_ID','y_next'],axis=1)
            final_test[cat_features] = final_test[cat_features].astype('category')
            
            final_test = scaler.fit_transform(final_test)
            final_predict = m.predict(final_test)
            final_predict_mean = final_predict.mean()

        return final_predict

In [15]:
def lgbm_odds_predict(team, predict_range, model_type):
    score = LGBM_score(lgbm_hitter_eda, team, predict_range, model_type)
    lose = LGBM_lose(lgbm_pitcher_eda, team, predict_range, model_type)

    lose = lose.run()
    score = score.run()

    score = np.array(score).mean()
    lose = np.array(lose).mean()
    win_rate_mean = score**2/(score**2 + lose**2)

    return win_rate_mean

# VAR

In [16]:
#library import
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
from IPython.display import display

import warnings
warnings.filterwarnings(action='ignore')

## EDA

### ERA

In [34]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def var_pitcher_eda(team_name):
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_2016 = data_games[0]
    data_games_2017 = data_games[1]
    data_games_2018 = data_games[2]
    data_games_2019 = data_games[3]
    data_games_2020 = data_games[4]
    data_games_year = pd.concat([data_games_2016,data_games_2017,data_games_2018,data_games_2019,data_games_2020])

    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_pitcher_2016 = data_team_pitcher[0]
    data_team_pitcher_2017 = data_team_pitcher[1]
    data_team_pitcher_2018 = data_team_pitcher[2]
    data_team_pitcher_2019 = data_team_pitcher[3]
    data_team_pitcher_2020 = data_team_pitcher[4]
    data_team_pitcher_year = pd.concat([data_team_pitcher_2016,data_team_pitcher_2017,data_team_pitcher_2018,
                                        data_team_pitcher_2019,data_team_pitcher_2020])

    data_team_pitcher_year = pd.merge(data_team_pitcher_year, data_games_year, how='left', on=['G_ID'])

    data_team_pitcher_year_team = data_team_pitcher_year[data_team_pitcher_year.T_ID == team_name]
    df = data_team_pitcher_year_team
    df= df[df.G_ID.isin(pithcer_outlier)==False]
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'G_ID'])
    
    df.reset_index(inplace=True, drop=True) #index 재정렬
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    df['TB_SC'] = encoder.fit_transform(df['TB_SC']) #이닝 초/말
    df['WLS'] = encoder.fit_transform(df['WLS']) #승패무 여부
        
    # GDAY_DS => Datetime type
    df['GDAY_DS'] = df['GDAY_DS'].astype('str') + (df['HEADER_NO']+1).astype('str')
        
    dt = []
    for i in df['GDAY_DS']:
        dt_ = datetime.strptime(i, '%Y%m%d%H')
        dt.append(dt_)
    df['GDAY_DS'] = pd.Series(dt)
    
    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('GDAY_DS')
    df = df.drop(columns=['T_ID'])
    

    # 경기당 방어율 column 생성
    df['ERA'] = df['ER'] * 27 / df['INN2']
    df['avg_ERA'] = df['ERA'].mean()
    df['ERA_trend'] = (df['ERA'] - df['avg_ERA']).astype(np.float16)
    df.drop(['avg_ERA'],axis=1,inplace=True)
    
    # Drop Categorical feature for 시계열 => coint 과정에서 singular matrix 발생
    df = df.drop(columns=['VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK', 'WLS', 'HOLD', 'HOME_KEY', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT'])
    
    df_year_team_name = df
    
    return df_year_team_name

### AVG

In [18]:
def var_hitter_eda(team_name):
    
    # team_pitcher data에 해당 경기의 해당 팀의 홈 어웨이 여부 column 추가
    data_games_2016 = data_games[0]
    data_games_2017 = data_games[1]
    data_games_2018 = data_games[2]
    data_games_2019 = data_games[3]
    data_games_2020 = data_games[4]
    data_games_year = pd.concat([data_games_2016,data_games_2017,data_games_2018,data_games_2019,data_games_2020])

    data_games_year = data_games_year[['G_ID', 'VISIT_KEY', 'HOME_KEY']]
    data_games_year = data_games_year.set_index('G_ID')
    
    data_team_hitter_2016 = data_team_hitter[0]
    data_team_hitter_2017 = data_team_hitter[1]
    data_team_hitter_2018 = data_team_hitter[2]
    data_team_hitter_2019 = data_team_hitter[3]
    data_team_hitter_2020 = data_team_hitter[4]
    data_team_hitter_year = pd.concat([data_team_hitter_2016,data_team_hitter_2017,data_team_hitter_2018,
                                        data_team_hitter_2019,data_team_hitter_2020])

    data_team_hitter_year = pd.merge(data_team_hitter_year, data_games_year, how='left', on=['G_ID'])

    data_team_hitter_year_team = data_team_hitter_year[data_team_hitter_year.T_ID == team_name]
    df = data_team_hitter_year_team
    df= df[df.G_ID.isin(hitter_outlier)==False]
    df['HOME_KEY'] = df['HOME_KEY'].map(lambda x: 1 if x == team_name else 0 )
    df = df.drop(columns = ['VISIT_KEY', 'G_ID'])
    
    df.reset_index(inplace=True, drop=True) #index 재정렬
    
    # LabelEncoding
    encoder = LabelEncoder()

    df['VS_T_ID'] = encoder.fit_transform(df['VS_T_ID']) #상대편
    
    # GDAY_DS => Datetime type
    df['GDAY_DS'] = df['GDAY_DS'].astype('str') + (df['HEADER_NO']+1).astype('str')
        
    dt = []
    for i in df['GDAY_DS']:
        dt_ = datetime.strptime(i, '%Y%m%d%H')
        dt.append(dt_)
    df['GDAY_DS'] = pd.Series(dt)
    
    # 시계열 데이터(경기당 방어율 계산)이기 때문에 경기 코드를 index로
    df = df.set_index('GDAY_DS')
    
    df = df.drop(columns=['T_ID'])
    
    df['H1']=df['HIT']-df['H2']-df['H3']-df['HR']
    #장타율
    df['SLG']=(df['HIT']+df['HP']+df['BB'])/(df['AB']+df['BB']+df['HP']+df['SF'])
    #출루율
    df['OBA']=(df['H1']+2*df['H2']+3*df['H3']+4*df['HR'])/df['AB']
    df['W_OPS']=0.57* df['SLG']+0.43*df['OBA']
    # 경기당 타율 column 생성
    df['AVG'] = df['HIT'] / df['AB']
    
    df['avg_AVG'] = df['AVG'].mean()
    df['AVG_trend'] = (df['AVG'] - df['avg_AVG']).astype(np.float16)
    df.drop(['avg_AVG'],axis=1,inplace=True)
    
    # Drop Categorical feature for 시계열 => coint 과정에서 singular matrix 발생
    df = df.drop(columns=['VS_T_ID', 'HEADER_NO', 'TB_SC', 'HOME_KEY'])
    
    df_year_team_name = df
    
    return df_year_team_name

## Modeling

In [19]:
class Var_automation:
    def __init__(self, df, target, predict_range, mode):
        super(Var_automation, self).__init__()
        
        self.mode = mode
        if self.mode=='train':
            self.df_set = df
            self.predict_range = predict_range
            self.train = self.df_set.iloc[:-self.predict_range]
            self.test = self.df_set.iloc[-self.predict_range:]
            self.target = target
        
        else:
            zero_np = np.zeros((predict_range,len(df.columns)))
            df_np = df.to_numpy()
            df_concat = np.concatenate((df_np, zero_np))
            self.df_set = pd.DataFrame(df_concat, columns = df.columns)

            self.predict_range = predict_range
            self.train = self.df_set.iloc[:-self.predict_range]
            self.test = self.df_set.iloc[-self.predict_range:]
            self.target = target

    
    def grangers_causation_matirx(self, test='ssr_chi2test', verbose=False): #인과관계 검정
        variables = self.df_set.columns
        maxlag = 8
        
        df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns = variables, index = variables)
        for column in df.columns:
            for index in df.index:
                test_result = grangercausalitytests(self.df_set[[index,column]], maxlag=maxlag, verbose=False)
                p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
                if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
                min_p_value = np.min(p_values)
                df.loc[index, column] = min_p_value
        df.columns = [var + '_x' for var in variables]
        df.index = [var + '_y' for var in variables]
        
        target_p_value = df.loc['{}_y'.format(self.target)]
        effective_var = []
        for i, v in enumerate(target_p_value):
            if v < 0.05:
                effective_var.append(df.index[i].split('_y')[0])
        
        return df, effective_var
    
    def cointegration_test(self, alpha=0.05): #공적분 검정
        df_grangers, var_grangers = self.grangers_causation_matirx()
        var_coint = var_grangers
        var_coint.append(self.target) 
        df = self.df_set[var_coint] 
        df_shape = df.shape
                
        try:
            out = coint_johansen(df, -1, 5)
        except:
            df = df + 0.00001*np.random.rand(df_shape[0],df_shape[1]) #Singular Matrix Error Solution
            out = coint_johansen(df, -1, 5)
            
        d = {'0.90': 0, '0.95': 1, '0.99': 2}
        traces = out.lr1
        cvts = out.cvt[:, d[str(1-alpha)]]
        def adjust(val, length= 6): return str(val).ljust(length)    
        
        dict = {'Name': [], 'Test Stat': [], '> C(95%)': [], 'Signif': []}
        effective_var = []

        for col, trace, cvt in zip(df.columns, traces, cvts):
            
            dict['Name'].append(adjust(col))
            dict['Test Stat'].append(adjust(round(trace,2), 9))
            dict['> C(95%)'].append(adjust(cvt, 8))
            dict['Signif'].append(trace > cvt)
            
            if trace>cvt:
                effective_var.append(col)
        
        df_coint = pd.DataFrame(dict)                
        
        return df_coint, effective_var
    
    def adfuller_test(self, signif=0.05, verbose=False): #단위근 검정
        df_coint, effective_var = self.cointegration_test()
        effective_var.append(self.target)
        
        df_train = self.train[effective_var]
        df_test = self.test[effective_var]
        
        def adjust(val, length=6): return str(val).ljust(length)
        
        adfuller_dict = {}
        stationary_var = []
        
        for name, series in df_train.items():
            name = series.name
            
            dict = {'adfuller_test var': [], 'Significance Level': [], 'Test Statistic': [], 'No. Lags Chosen': []}
    
            r = adfuller(series, autolag = 'AIC')
            output = {'test_static': round(r[0], 4), 'pvalue': round(r[1], 4), 'n_lags': round(r[2], 4), 'n_obs': r[3]}
            p_value = output['pvalue']
            
            dict['adfuller_test var'].append(name)
            dict['Significance Level'].append(signif)
            dict['Test Statistic'].append(output['test_static'])
            dict['No. Lags Chosen'].append(output['n_lags'])
            
            for key, val in r[4].items():
                dict['Critical value {}'.format(adjust(key))] = round(val, 3)
                
            if p_value <= signif:
                dict['P-Value'] = '{} => Series is Stationary'.format(p_value)
                stationary_var.append(name)
            else:
                dict['P-Value'] = '{} => Series is Non-Stationary'.format(p_value)
            
            adfuller_dict[name] = dict
        
        return adfuller_dict, stationary_var
    
    def aic_test(self, lag=8):
        dict_, effective_var = self.adfuller_test()
        df_train = self.train[effective_var]
        
        model = sm.tsa.VAR(df_train)
        
        dict = {}
        for i in range(1, lag+1, 1):
            try:
                result = model.fit(i)
                dict[i] = result.aic
            except:
                continue
        
            
        sorted_dict = sorted(dict.items(), key = lambda x: x[1])
        min_aic = sorted_dict[0]
        
        min_lag = min_aic[0]
        min_lag_aic = min_aic[1]
        
        return min_lag, min_lag_aic, model, effective_var
    
    def var_fit(self, model_summary=False):
        min_lag, min_lag_aic, model, effective_var = self.aic_test()
        model_fitted = model.fit(min_lag)
        
        if model_summary:
            model_fitted.summary()
            
        return model_fitted, effective_var
    
    def durbin_watson_test(self):
        model_fitted, effective_var = self.var_fit()
        df_train = self.train[effective_var]
        out = durbin_watson(model_fitted.resid)
        
        durbin_dict = {}
        for col, val in zip(df_train.columns, out):
            durbin_dict[col] = round(val,2)
        
        return durbin_dict, model_fitted, effective_var
    
    def run(self, displaying=False): #nobs=20, 
        model_fitted, effective_var = self.var_fit()
        df_train = self.train[effective_var]
        df_test = self.test[effective_var]
        
        lag_order = model_fitted.k_ar
        forecast_input = df_train.values[-lag_order:]
        
        fc = model_fitted.forecast(y=forecast_input, steps=self.predict_range)
        df_forecast = pd.DataFrame(fc, index = df_test.index, columns=effective_var)
        
        #display
        if displaying:
            
            display(df_forecast)
        
            fig,axes = plt.subplots(nrows=int(len(effective_var)/2), ncols=2, dpi=150, figsize=(10,10))
            for i, (col,ax) in enumerate(zip(effective_var, axes.flatten())):
                df_test[col].plot(legend=True, ax=ax);
                df_forecast[col].plot(legend=True, ax=ax).autoscale(axis='x', tight=True)

                ax.set_title(col + ": Forecast vs Actuals")
                ax.xaxis.set_ticks_position('none')
                ax.yaxis.set_ticks_position('none')
                ax.spines["top"].set_alpha(0)
                ax.tick_params(labelsize=6)

            plt.tight_layout()
        true_mean = self.test[self.target].mean()
        predict_mean = df_forecast[self.target].mean()
        
        return df_forecast[self.target].mean()

# LSTM

In [20]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


## EDA

In [21]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

## Modeling

In [22]:
class Lstm_model:
    
    def __init__(self, df, target, look_back, predict_range, mode):
        self.df = df[[target]]
        self.target = target
        self.look_back = 5
        self.predict_range = predict_range
        self.mode = mode
        
    def univariate(self):
        if self.mode == 'train':
            signal_data =self.df
            # 데이터 전처리
            scaler = MinMaxScaler(feature_range=(0, 1))
            signal_data = scaler.fit_transform(signal_data)

            # 데이터 분리
            train = signal_data[0:-self.predict_range]
            test = signal_data[-self.predict_range:]

            # 데이터셋 생성
            x_train, y_train = create_dataset(train, self.look_back)
            x_test, y_test = create_dataset(test, self.look_back)

            x_train = np.reshape(x_train,(x_train.shape[0],self.look_back,1)) #(size, timestamp,feature)
            x_test = np.reshape(x_test,(x_test.shape[0],self.look_back,1))

            return x_train, x_test, y_train, y_test,scaler, self.look_back
        
        else:
            signal_data = self.df
            # 데이터 전처리

            scaler = MinMaxScaler(feature_range=(0, 1))
            signal_data = scaler.fit_transform(signal_data)
            last_data = signal_data[-self.look_back:]
            # 데이터셋 생성
            x_train, y_train = create_dataset(signal_data, self.look_back)

            x_train = np.reshape(x_train,(x_train.shape[0],self.look_back,1)) #(size, timestamp,feature)

            return x_train, y_train,scaler, self.look_back, last_data
    
    def run_stateful(self, x_train, x_test, y_train, y_test, look_back, scaler,box1):
        model = Sequential()
        model.add(LSTM(box1, batch_input_shape=(1, look_back, 1), stateful=True))
        model.add(Dropout(0.1))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

        model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop])
        score = model.evaluate(x_test,y_test,batch_size=1)
        pre =model.predict(x_test,batch_size=1) #모든 batch_size 바꾸면 error
        ans = y_test

        return score, pre, ans
    
    def run_stateful_ans(self, x_train,  y_train, look_back,box):
        model = Sequential()
        model.add(LSTM(box, batch_input_shape=(1, look_back, 1), stateful=True))
        model.add(Dropout(0.1))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

        model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop])
        return model
    
    def run(self):
        if self.mode == 'train':
            x_train, x_test, y_train, y_test,scaler,look_back = self.univariate()
            score, pre, ans = self.run_stateful_stack(x_train, x_test, y_train, y_test,look_back, scaler,5)
            sc_pre=scaler.inverse_transform(pre)
            sc_ans=scaler.inverse_transform(ans.reshape(ans.shape[0],1))
            predict_mean = sc_pre.mean()
            real_mean = sc_ans.mean()
        
            return predict_mean, real_mean
    
        else:
            x_train, y_train,scaler, look_back,last_data = self.univariate()
            model = self.run_stateful_ans(x_train,  y_train, look_back,5)
            pred_date = self.predict_range
            seq_in = [i[0] for i in last_data]
            seq_out = []
            for i in range(pred_date):
                sample_in = np.reshape(seq_in, (1, look_back,1)) # batch_size, feature
                pred_out = model.predict(sample_in)[0][0]
                seq_out.append(pred_out)
                seq_in.append(pred_out)
                seq_in.pop(0)
            ans = scaler.inverse_transform(np.array(seq_out).reshape(pred_date,1))
            ans_mean=ans.mean()
            return ans_mean
        

In [23]:
class dnn_score:
    def __init__(self,AVG,ERA,team,hiter,pitcher,mode):
        self.AVG =AVG
        self.ERA =ERA
        self.team = team
        self.hiter = hiter
        self.pitcher = pitcher
        self.mode = mode
        
    def prepro(self, hiter, pitcher): #전처리
        
        hiter['AVG']= hiter['HIT']/hiter['AB']
        hiter["y_m"]=hiter.GDAY_DS.astype(str).str[2:6]
        hiter=hiter[["T_ID","AVG",'y_m']]
    
        pitcher["ERA"] = 27*pitcher.ER/pitcher.INN2
        pitcher.WLS = pitcher.WLS.map({'W':1,'D':0.5,"L":0})
        pitcher["y_m"]=pitcher.GDAY_DS.astype(str).str[2:6]
        pitcher=pitcher[["T_ID","WLS","ERA","y_m"]]
    
        return hiter, pitcher
    
    def make_data(self):
        hiter, pitcher = self.prepro(self.hiter, self.pitcher)
        team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
        df_mean=pd.DataFrame()
        code=0
        for team in team_list:
            code +=0.1
            pitcher_team= pitcher[pitcher.T_ID==team]
            hiter_team=hiter[hiter.T_ID==team]
            pitcher_mean = pitcher_team.groupby(['y_m']).mean()
            pitcher_mean["count"] =pitcher_team.groupby(['y_m']).count().T_ID
            hiter_mean = hiter_team.groupby(['y_m']).mean()
            df = pd.concat([pitcher_mean, hiter_mean], axis=1)
            df["team_code"] = code
            df_mean=pd.concat([df_mean,df])
    
        df_mean.reset_index(inplace=True, drop=True)
        df_mean = df_mean[df_mean["count"] > 9]
        df_mean.reset_index(inplace=True, drop=True)
        del df_mean["count"]
        
        sc_era = MinMaxScaler()
        sc_avg = MinMaxScaler()
        df_mean["AVG"]=sc_avg.fit_transform(df_mean[["AVG"]])
        df_mean["ERA"]=1-sc_era.fit_transform(df_mean[["ERA"]])

        df_mean=df_mean.sample(frac=1).reset_index(drop=True)

        if self.mode == 'train':
            x_train=df_mean.iloc[:240,1:]
            y_train=df_mean.iloc[:240,0]
            x_test=df_mean.iloc[240:,1:]
            y_test=df_mean.iloc[240:,0]
            return x_train, y_train,x_test,y_test, sc_era,sc_avg
        
        else:
            x_train=df_mean.iloc[:,1:]
            y_train=df_mean.iloc[:,0]
            
            return x_train, y_train, sc_era,sc_avg
    
    def run(self):
        if self.mode == "train":
            x_train, y_train,x_test,y_test, sc_era,sc_avg = self.make_data()
        else:
            x_train, y_train, sc_era,sc_avg = self.make_data()
        model=Sequential()
        model.add(Dense(20,activation='relu',input_dim=3))
        model.add(Dense(5,activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
        model.fit(x_train, y_train, epochs=100, validation_split=0.15,callbacks=[early_stop])
        
        team_map = {'WO':0.1,'OB':0.2,'NC':0.3,'SK':0.4,'LG':0.5,'KT':0.6,'LT':0.7,'HT':0.8,'SS':0.9,'HH':1.0}

        if self.mode == 'train':
            evaluate = model.evaluate(x_test, y_test)
            pre=model.predict(x_test)
            ny_test=y_test.values.reshape(y_test.shape[0],1)
            print(evaluate)
            print(np.concatenate((pre,ny_test),axis=1))
        else:
            final_df=pd.DataFrame({"AVG":[self.AVG],"ERA":[self.ERA],"team_code":[team_map[self.team]]})
            final_df["AVG"] = sc_avg.transform(final_df[["AVG"]])
            final_df["ERA"] = 1-sc_era.transform(final_df[["ERA"]])
            win_rate= model.predict(final_df)
        
            return win_rate[0][0]

# Ensemble

In [24]:
def ERA_ensemble(team, predict_range):
    
    var_df = var_pitcher_eda(team)
    
    obj1 = LGBM_pitcher(lgbm_pitcher_eda, team, predict_range, 'lgbm')
    model1_predict = obj1.run()
    
    obj2 = LGBM_pitcher(lgbm_pitcher_eda, team, predict_range, 'rf')
    model2_predict = obj2.run()
    
    obj3 = LGBM_pitcher(lgbm_pitcher_eda, team, predict_range, 'SVR')
    model3_predict = obj3.run()
    
    obj4 = Var_automation(var_df, 'ERA', predict_range, mode='inference')
    model4_predict = obj4.run()
    
    obj5 = Lstm_model(var_df, 'ERA', 5, predict_range, mode='inference')
    model5_predict = obj5.run()
    
    list_2020 = []
    for i,v in enumerate(var_df.index):
        if str(v).split('-')[0] == '2020':
            list_2020.append(i)
    var_df_2020 = var_df.iloc[list_2020[0]:,:]
    
    past_mean = var_df_2020['ERA'].mean() #해당 연도의 평균만
    
    predict_list = [model1_predict, model2_predict, model3_predict, model4_predict, model5_predict, past_mean]
    
    final_predict = (model1_predict+model2_predict+model3_predict+model4_predict+model5_predict+past_mean) / 6
    
    return predict_list, final_predict
    
    

In [25]:
def AVG_ensemble(team, predict_range):
    
    var_df = var_hitter_eda(team)
    
    obj1 = LGBM_hitter(lgbm_hitter_eda, team, predict_range, 'lgbm')
    model1_predict = obj1.run()
    
    obj2 = LGBM_hitter(lgbm_hitter_eda, team, predict_range, 'rf')
    model2_predict = obj2.run()
    
    obj3 = LGBM_hitter(lgbm_hitter_eda, team, predict_range, 'SVR')
    model3_predict = obj3.run()
    
    obj4 = Var_automation(var_df, 'AVG', predict_range, mode='inference')
    model4_predict = obj4.run()
    
    obj5 = Lstm_model(var_df, 'AVG', 5, predict_range, mode='inference')
    model5_predict = obj5.run()
    
    list_2020 = []
    for i,v in enumerate(var_df.index):
        if str(v).split('-')[0] == '2020':
            list_2020.append(i)
    var_df_2020 = var_df.iloc[list_2020[0]:,:]
    
    past_mean = var_df_2020['AVG'].mean() #해당 연도의 평균만
    
    predict_list = [model1_predict, model2_predict, model3_predict, model4_predict, model5_predict, past_mean]
    
    final_predict = (model1_predict+model2_predict+model3_predict+model4_predict+model5_predict+past_mean) / 6
    
    return predict_list, final_predict

In [26]:
def odds_ensemble(team, predict_range):
    
    var_hitter_df = var_hitter_eda(team)
    var_pitcher_df = var_pitcher_eda(team)
    
    hitter_df = pd.concat([data_team_hitter[0],data_team_hitter[1],data_team_hitter[2],data_team_hitter[3],data_team_hitter[4]])
    pitcher_df = pd.concat([data_team_pitcher[0],data_team_pitcher[1],data_team_pitcher[2],data_team_pitcher[3],data_team_pitcher[4]])
    
    model1_predict = lgbm_odds_predict(team, predict_range, 'lgbm')
    model2_predict = lgbm_odds_predict(team, predict_range, 'rf')
    model3_predict = lgbm_odds_predict(team, predict_range, 'SVR')
    
    lstm_ERA_obj = Lstm_model(var_hitter_df, 'AVG', 5, predict_range, mode='inference')
    lstm_ERA= lstm_ERA_obj.run()
    
    lstm_AVG_obj = Lstm_model(var_pitcher_df, 'ERA', 5, predict_range, mode='inference')
    lstm_AVG = lstm_AVG_obj.run()
    
    model4_obj = dnn_score(lstm_ERA, lstm_AVG, team, hitter_df ,pitcher_df,mode='inference')
    model4_predict = model4_obj.run()
    
    predict_list = [model1_predict, model2_predict, model3_predict, model4_predict]    
        
    final_predict = (model1_predict+model2_predict+model3_predict+model4_predict) / 4
    
    return predict_list, final_predict

# Prediction

In [35]:
team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
experiment_dict = {'방어율':[], '타율':[], '승률':[]}

#WO
era_predict_list, era_final_predict = ERA_ensemble('WO', 20)
experiment_dict['final_ERA'].append(era_final_predict)

avg_predict_list, avg_final_predict = AVG_ensemble('WO', 20)
experiment_dict['final_AVG'].append(avg_final_predict)

odds_predict_list, odds_final_predict = odds_ensemble('WO', 20)
experiment_dict['final_odds'].append(odds_final_predict)

#OB
era_predict_list, era_final_predict = ERA_ensemble('OB', 25)
experiment_dict['final_ERA'].append(era_final_predict)

avg_predict_list, avg_final_predict = AVG_ensemble('OB', 25)
experiment_dict['final_AVG'].append(avg_final_predict)

odds_predict_list, odds_final_predict = odds_ensemble('OB', 25)
experiment_dict['final_odds'].append(odds_final_predict)

#NC
era_predict_list, era_final_predict = ERA_ensemble('NC', 28)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('NC', 28)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('NC', 28)
experiment_dict['final_odds'].append(odds_final_predict)


#SK
era_predict_list, era_final_predict = ERA_ensemble('SK', 24)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('SK', 24)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('SK', 24)
experiment_dict['final_odds'].append(odds_final_predict)


#LG
era_predict_list, era_final_predict = ERA_ensemble('LG', 25)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('LG', 25)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('LG', 25)
experiment_dict['final_odds'].append(odds_final_predict)

#KT
era_predict_list, era_final_predict = ERA_ensemble('KT', 27)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('KT', 27)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('KT', 27)
experiment_dict['final_odds'].append(odds_final_predict)

#LT 
era_predict_list, era_final_predict = ERA_ensemble('LT', 25)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('LT', 25)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('LT', 25)
experiment_dict['final_odds'].append(odds_final_predict)

#HT
era_predict_list, era_final_predict = ERA_ensemble('HT', 29)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('HT', 29)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('HT', 29)
experiment_dict['final_odds'].append(odds_final_predict)

#SS
era_predict_list, era_final_predict = ERA_ensemble('HT', 26)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('HT', 26)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('HT', 26)
experiment_dict['final_odds'].append(odds_final_predict)

#HH
era_predict_list, era_final_predict = ERA_ensemble('HH', 26)
experiment_dict['final_ERA'].append(era_final_predict)
avg_predict_list, avg_final_predict = AVG_ensemble('HH', 26)
experiment_dict['final_AVG'].append(avg_final_predict)
odds_predict_list, odds_final_predict = odds_ensemble('HH', 26)
experiment_dict['final_odds'].append(odds_final_predict)

experiment_df = pd.DataFrame(experiment_dict, index=team_list)
display(experiment_df)


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    9.1s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   32.6s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done 480 out of 480 | elapsed:  2.2min finished



Epoch 1/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0351
Epoch 2/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0327
Epoch 3/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0323
Epoch 4/100
691/691 [==============================] - 2s 4ms/step - loss: 0.0322
Epoch 5/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0320
Epoch 6/100
691/691 [==============================] - 2s 4ms/step - loss: 0.0319
Epoch 7/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0318
Epoch 8/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0316
Epoch 9/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0318
Epoch 10/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0316
Epoch 11/100
691/691 [==============================] - 2s 3ms/step - loss: 0.0317
Epoch 00011: early stopping
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    6.0s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   35.7s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:   55.1s
[Parallel(n_jobs=10)]: Done 480 out of 480 | elapsed:  1.1min finished


Epoch 1/100
690/690 [==============================] - 3s 5ms/step - loss: 0.0639
Epoch 2/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0448
Epoch 3/100
690/690 [==============================] - 2s 4ms/step - loss: 0.0420
Epoch 4/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0416
Epoch 5/100
690/690 [==============================] - 2s 4ms/step - loss: 0.0394
Epoch 6/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0390
Epoch 7/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0401
Epoch 8/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0399
Epoch 9/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0387
Epoch 10/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0384
Epoch 11/100
690/690 [==============================] - 3s 4ms/step - loss: 0.0374
Epoch 12/100
690/690 [==============================] - 2s 4ms/step - loss: 0.0381
Epoch 13/100


...

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    7.8s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   40.1s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done 480 out of 480 | elapsed:  2.2min finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    6.6s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   35.7s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done 480 out of 480 | elapsed:  2.3min finished


Epoch 1/100
686/686 [==============================] - 5s 7ms/step - loss: 0.0452
Epoch 2/100
686/686 [==============================] - 3s 5ms/step - loss: 0.0338
Epoch 3/100
686/686 [==============================] - 3s 5ms/step - loss: 0.0304
Epoch 4/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0304
Epoch 5/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0290
Epoch 6/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0288
Epoch 7/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0286
Epoch 8/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0278
Epoch 9/100
686/686 [==============================] - 4s 5ms/step - loss: 0.0278
Epoch 10/100
686/686 [==============================] - 4s 6ms/step - loss: 0.0283
Epoch 11/100
686/686 [==============================] - 4s 6ms/step - loss: 0.0276
Epoch 12/100
686/686 [==============================] - 4s 6ms/step - loss: 0.0263
Epoch 13/100


,final_ERA,final_AVG,final_odds
WO,4.567643,0.270651,0.550921
OB,4.506169,0.284679,0.547750
NC,4.643764,0.278741,0.591960
SK,5.427254,0.257827,0.410641
LG,4.627730,0.274902,0.492944
KT,4.918385,0.272521,0.502009
LT,5.027182,0.271161,0.488329
HT,5.307015,0.277560,0.459590
SS,5.360450,0.275785,0.432444
HH,5.200824,0.256868,0.393560


In [44]:
#HH
prediction_by_models={'ERA_LGBM':[], 'ERA_RF':[], 'ERA_SVR':[], 'ERA_VAR':[], 'ERA_LSTM':[], 'ERA_2020':[],
                     'AVG_LGBM':[], 'AVG_RF':[], 'AVG_SVR':[], 'AVG_VAR':[], 'AVG_LSTM':[], 'AVG_2020':[],
                     'odds_LGBM':[], 'odds_RF':[], 'odds_SVR':[], 'odds_DNN':[]}

prediction_by_models['ERA_LGBM'].append(era_predict_list[0])
prediction_by_models['ERA_RF'].append(era_predict_list[1])
prediction_by_models['ERA_SVR'].append(era_predict_list[2])
prediction_by_models['ERA_VAR'].append(era_predict_list[3])
prediction_by_models['ERA_LSTM'].append(era_predict_list[4])
prediction_by_models['ERA_2020'].append(era_predict_list[5])
prediction_by_models['AVG_LGBM'].append(avg_predict_list[0])
prediction_by_models['AVG_RF'].append(avg_predict_list[1])
prediction_by_models['AVG_SVR'].append(avg_predict_list[2])
prediction_by_models['AVG_VAR'].append(avg_predict_list[3])
prediction_by_models['AVG_LSTM'].append(avg_predict_list[4])
prediction_by_models['AVG_2020'].append(avg_predict_list[5])
prediction_by_models['odds_LGBM'].append(odds_predict_list[0])
prediction_by_models['odds_RF'].append(odds_predict_list[1])
prediction_by_models['odds_SVR'].append(odds_predict_list[2])
prediction_by_models['odds_DNN'].append(odds_predict_list[3])

prediction_by = pd.DataFrame(prediction_by_models, index=['HH'])
display(prediction_by)

,ERA_LGBM,ERA_RF,ERA_SVR,ERA_VAR,ERA_LSTM,ERA_2020,AVG_LGBM,AVG_RF,AVG_SVR,AVG_VAR,AVG_LSTM,AVG_2020,odds_LGBM,odds_RF,odds_SVR,odds_DNN
HH,5.297633,5.074899,5.470508,5.264835,4.846072,5.250997,0.246836,0.255581,0.264975,0.264095,0.275041,0.234677,0.333691,0.383767,0.402271,0.454513


In [37]:
experiment_df.to_csv('BigContact_result.csv', encoding='utf-8-sig')

In [45]:
prediction_by.to_csv('BigContact_result_by_models.csv', encoding='utf-8-sig')